In [5]:
%load_ext autoreload
%autoreload 1

%matplotlib inline

import xarray as xr
import pandas as pd
import datetime
import matplotlib.dates as mdates
import gsw
import glidertools as gt
from tqdm.notebook import tqdm_notebook as tqdm
import geomag
from cmocean import cm as cmo
import matplotlib.image as mpimg
import matplotlib as mpl
import cartopy.crs as ccrs
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,AutoMinorLocator)
from tqdm.notebook import tqdm_notebook as tqdm
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import matplotlib.ticker as mticker
import sys
from datetime import date

sys.path.append('/Volumes/MASSIVEUNIT/Work/SOCHIC_2022/sailbuoy_processing/code/')

%aimport dl_tools

font = {'family' : 'Avenir',
        'weight' : 'normal',
        'size'   : 30}

mpl.rc('font', **font)

mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.size'] = 6
mpl.rcParams['xtick.minor.width'] = 1

def rot_ticks(axs,rot,ha):
    for xlabels in axs.get_xticklabels():
                xlabels.set_rotation(rot)
                xlabels.set_ha(ha)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SB Kringla

Load and prep the data.txt from the datalogger.

In [19]:
path = '/Volumes/MASSIVEUNIT/Work/SOCHIC_2022/Sailbuoy/data/DATA.TXT'

ds = dl_tools.load_data(path)                                        # Loading the data
ds = dl_tools.fix_standard_attrs(ds)                                 # Setting attributes and names for the standard variables
ds = dl_tools.fix_airmar(ds)                                         # Setting attributes and names for the Airmar variables
ds = dl_tools.fix_dcps(ds)                                           # Setting attributes and names for the DCPS variables
ds = dl_tools.fix_aadi(ds)                                           # Setting attributes and names for the AADI Conductivity variables
ds = ds.sortby('time').sel(time=slice('2022-01-10','2022-07-17'))    # Sorting in time, and selecting the time for the mission

Checking lines in input file:   0%|          | 0/20213 [00:00<?, ?it/s]

20208 good measurements found in input file
5 bad lines found in input file



#### Assigning the metadata

In [20]:
ds.attrs['title']                   = 'SB Kringla Datalogger'
ds.attrs['summary']                 = 'Datalogger output from Sailbuoy Kringla'
ds.attrs['conventions']             = 'CF-1.6, ACDD-1.3'
ds.attrs['creator_name']            = 'Johan Edholm'
ds.attrs['platform']                = 'Uncrewed surface vehicle'
ds.attrs['instrument_name']         = 'Kringla'
ds.attrs['instrument_id']           = '1812'
ds.attrs['institution']             = 'University of Gothenburg'
ds.attrs['creator_email']           = 'johan.edholm@gu.se'
ds.attrs['area']                    = 'Southern Ocean'
ds.attrs['project']                 = 'SO-CHIC'
ds.attrs['id']                      = 'SB Kringla Datalogger SO-CHIC'
ds.attrs['instrument']              = 'Offshore Sensing AS Sailbuoy'
ds.attrs['owner']                   = 'University of Gothenburg'
ds.attrs['contact']                 = 'sebastiaan.swart@marine.gu.se'
ds.attrs['processing_date']         = str(date.today())
ds.attrs['processing_level']        = '2'
ds.attrs['time_coverate_start']     = str(ds.time[0].values)
ds.attrs['time_coverate_end']       = str(ds.time[-1].values)
ds.attrs['geospatial_lat_min']      = str(ds.latitude.min().values)
ds.attrs['geospatial_lat_max']      = str(ds.latitude.max().values)
ds.attrs['geospatial_lon_min']      = str(ds.longitude.min().values)
ds.attrs['geospatial_lon_max']      = str(ds.longitude.max().values)

In [23]:
ds


<xarray.Dataset>
Dimensions:         (time: 19512)
Coordinates:
  * time            (time) datetime64[ns] 2022-01-10T11:09:11 ... 2022-07-17T...
Data variables: (12/24)
    latitude        (time) float64 -63.3 -63.3 -63.3 ... -53.68 -53.68 -53.68
    longitude       (time) float64 2.448 2.448 2.448 ... 0.1208 0.1208 0.1208
    ttff            (time) float64 11.0 11.0 12.0 11.0 ... 306.0 306.0 306.0
    count           (time) float64 0.0 1.0 2.0 3.0 4.0 ... 1.0 2.0 3.0 4.0 5.0
    commands        (time) float64 0.0 0.0 0.0 0.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    tries           (time) float64 0.0 1.0 1.0 1.0 1.0 ... 1.0 3.0 3.0 3.0 3.0
    ...              ...
    ssc             (time) float64 nan nan nan nan ... 0.05111 0.05013 0.04928
    sst             (time) float64 nan nan nan nan ... -2.35 -2.004 -1.306
    dcps_flag       (time) float64 nan nan nan nan 0.0 ... nan nan nan nan nan
    dcps_time       (time) float64 nan nan nan nan 4.0 ... nan nan nan nan nan
    dcps_speed      (time) float64 nan nan nan nan 1.501 ... nan nan nan nan nan
    dcps_dir        (time) float64 nan nan nan nan 167.5 ... nan nan nan nan nan
Attributes: (12/23)
    title:                SB Kringla Datalogger
    summary:              Datalogger output from Sailbuoy Kringla
    conventions:          CF-1.6, ACDD-1.3
    creator_name:         Johan Edholm
    platform:             Uncrewed surface vehicle
    instrument_name:      Kringla
    ...                   ...
    time_coverate_start:  2022-01-10T11:09:11.000000000
    time_coverate_end:    2022-07-17T15:30:00.000000000
    geospatial_lat_min:   -66.519891
    geospatial_lat_max:   -53.47539
    geospatial_lon_min:   -1.327634
    geospatial_lon_max:   6.139851